In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import sys
import os 
import pandas as pd
import numpy as np
from collections import defaultdict


sys.path.append('utils/')
sys.path.append('feature/')
import config
from w2v import load_my_train_w2v,load_pre_train_w2v
from CutWord import cut_word
from feat_gen import *
from Feats import *

if config.use_pre_train:
    vocab, embed_weights = load_pre_train_w2v(config.origin_csv)
else:
    vocab, embed_weights = load_my_train_w2v(config.origin_csv)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.405 seconds.
Prefix dict has been built succesfully.


(1299, 256)
save embed_weights!
(1299, 256)
save embed_weights!


In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True
session = tf.Session(config=tfconfig)
import sys
import numpy as np
import pandas as pd
import pickle
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras import backend as K
from keras.engine.topology import Layer
from keras.backend.tensorflow_backend import set_session
import time
from tensorflow.contrib import learn
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.callbacks import TensorBoard
from keras.callbacks import Callback

sys.path.append('models')
from CNN import cnn_v1, cnn_v2, model_conv1D_
from ESIM import esim, decomposable_attention
from ABCNN import ABCNN
sys.path.append('utils/')
sys.path.append('feature/')
import config
from Feats import load_final_train_df
from help import score, train_batch_generator, train_batch_generator3,train_test, get_X_Y_from_df


Using TensorFlow backend.


In [3]:
df1 = human_feats(config.origin_csv)

cut done
magic_f1 done
megic_f2 done


TypeError: wordmatch1() takes at least 2 arguments (1 given)

In [ ]:
data = load_final_train_df(config.origin_csv)
train, dev = train_test(data)
x_train, y_train = get_X_Y_from_df(train, config.data_augment)
x_dev, y_dev = get_X_Y_from_df(dev, False)

In [ ]:
y_train.argmax(axis=1)

In [ ]:
x_train[2]

In [ ]:
import pandas as pd 
import gc
import xgboost as xgb
from math import floor
from time import time





# Train model
X_train = x_train[2][:,6:]
print(X_train.shape)
Y_train = y_train.argmax(axis=1)

X_valid = x_dev[2][:,6:]
Y_valid = y_dev.argmax(axis=1)

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'


d_train = xgb.DMatrix(X_train, label=Y_train)
d_valid = xgb.DMatrix(X_valid, label=Y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

print('train first model....')
bst = xgb.train(params, d_train, 50000, watchlist,early_stopping_rounds=50, verbose_eval=10)
print(time()-start_time)

# This is used to determine the best round to stop on. This is not the ideal traning method,
# but it does allow training the model on the entire data set, which gave some gains over
# using a only a train/test method + early stopping.
n_iter = bst.best_iteration




In [ ]:
pred = bst.predict(d_valid)
pred = [int(x<0.5) for x in pred]

In [ ]:
pre, rec, f1 = score(Y_valid, pred)

print('p r f1 ', pre, rec, f1)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

def score(label, pred):

    if len(label.shape) == 1:
        p = pred
        l = label
    else:
        p = np.argmax(pred, axis=1)
        l = np.argmax(label, axis=1)

    #print(confusion_matrix(l, p).view())
    pre_score = precision_score(l, p, pos_label=1, average='binary')
    rec_score = recall_score(l, p, pos_label=1, average='binary')
    f_score = f1_score(l, p)
    return pre_score, rec_score, f_score